In [24]:
import sys
import pandas as pd
import numpy as np
import os
#import matplotlib.pyplot as plt
#plt.style.use('ggplot')
#%matplotlib inline
from geopy.geocoders import Nominatim
geolocator = Nominatim(scheme='http',timeout=5)

In [2]:
df = pd.read_csv('/Users/suda77/Desktop/Score - Global LGBTQI laws dataset.csv')

In [ ]:
df.info()
df.head()
#sys.version

In [4]:
df['Punish']=df.apply(lambda r: (r['Penalising_text_Sexual act']+ r['Penalising_text_Sodomy']+ r['Penalising_text_Against nature']+ r['Penalising_text_Buggery'] +r['Penalising_text_Indecency'])*(1*r['MaxSentences_1Monthto2Years']+ 2*r['MaxSentences_3Yearsto7Years']+ 3*r['MaxSentences_8Yearsto13Years']+ 4*r['MaxSentences_14YearstoLife'] +10*r['MaxSentences_Death']), axis=1)

df.iloc[:,20] = df.iloc[:,20].apply(lambda v: 0.5 if v == 0 else 1)
df.iloc[:,14] = df.iloc[:,14].apply(lambda v: 1.5 if v == 1 else 0)


In [5]:
df.iloc[:10,:]
df['Score'] = df.apply(lambda r: 2*r['Legal_All_genders'] + 1*r['Ageofconsent_Equal'] -1*r['Ageofconsent_Unequal']-1*r['Illegal_Male']-1*r['Illegal_Female']-1*r['Arrests']*r['Punish']*r['PromotionorMorality_Morality_code']-1*r['PromotionorMorality_Penal_code']-1*r['BanonNGOs_Yes']+1*r['NRHISOinclusive_Yes']-1*r['NRHISONo']+5*r['Protection_Constitution']+1*r['Protection_Employment']+1*r['Protection_Other']+1*r['Protection_HateCrime']+1*r['Protection_Incitemenrt']+1*r['Protection_ConversionTherapyBan']+5*r['Relationshiprecognition_Marriage']+2*r['Relationshiprecognition_Civilrecognition']+2*r['Relationshiprecognition_Jointadoption']+2*r['Relationshiprecognition_2ndparentadoption'], axis=1)

In [6]:
df.loc[df['Country']=='Egypt','Score']= -3
df.loc[df['Country']=='Indonesia','Score']= 1
df.loc[df['Country']=='Jordan','Score']= 0
df.loc[df['Country']=='Qatar','Score']= -16

In [7]:
df.iloc[:10,:]
df.to_csv('/Users/suda77/Desktop/Score_Global_LGBTQI_laws_adjusted.csv', index=False)

In [12]:
df1 = pd.read_csv('/Users/suda77/Desktop/2017_Requests_by_nationality.csv')
df2 = pd.read_csv('/Users/suda77/Desktop/2017_travel_complete.csv')

In [ ]:
df1['Combine'] = df1['Nationality'].apply(lambda v: v[:4])
df3 = df1.groupby('Combine')['Requests received (excluding duplicates)'].sum().to_frame()
df3

In [ ]:
df2['Combine'] =df2['Country'].apply(lambda v: v[:4])
df4 = df2.groupby('Combine')['Individuals traveled'].sum().to_frame()
df4

In [ ]:
df5 = pd.concat([df3, df4], axis=1, join='outer')
df5.iloc[28:66,]

In [22]:
df5.to_csv('/Users/suda77/Desktop/Travel_Request_and_Complete.csv', index=True, index_label='Country')
#match-up 4-letter abreviation to full country name uinsg excel vlookup

In [27]:
df6 = pd.read_csv('/Users/suda77/Desktop/Travel_Request_and_Complete.csv')
df6['latitude'] = df6['Country'].apply(lambda v: geolocator.geocode(v).latitude)
df6['longitude'] = df6['Country'].apply(lambda v: geolocator.geocode(v).longitude)

In [ ]:
df6.info()
df6 = df6.fillna(0)
df6.loc[:,'Requests received'] = df6.loc[:,'Requests received'].astype(int)
df6.loc[:,'Individuals traveled'] = df6.loc[:,'Individuals traveled'].astype(int)
df6.to_csv('/Users/suda77/Desktop/Travel_Request_and_Complete2.csv', index=False)